In [ ]:
pip install nfl_data_py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.2 MB/s eta 0:00:00
  Created wheel for nfl_data_py: filename=nfl_data_py-0.3.0-py3-none-any.whl size=25103 sha256=3387dad4262eee18b9ab1061ff74ba5ebdbe5f1b8433fb9245e011c131a51629
  Stored in directory: /root/.cache/pip/wheels/0d/dd/64/ac11acfab32b58680881c80ff67db2ca9509f830e4782ecc07
Successfully built nfl_data_py
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

#modeling
from sklearn.svm import SVR, LinearSVC
from sklearn.linear_model import LinearRegression,LassoCV,Ridge,Lasso,ElasticNet
from sklearn.tree import DecisionTreeRegressor

#eval
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_selection import RFE, SelectKBest, chi2, f_regression, VarianceThreshold

# Pipeline Outline and Pending Items

In [ ]:
#how will pipeline work?

#1. function to extract all relevant dfs from nfl data py, return all dfs --> done

#2. function to aggregate pbp data, return multiple dfs: fantasy player results, team offense results, team defense results --> done

#3a. merge pbp with weekly player df (player id for pbp **retain game id to join to weekly df later if needed --> done
#3b. calculate different fantasy point totals in player df (standard, 0.5PPR and PPR) --> done
#3c. merge weekly player and team offense df to calc share stats --> merge done, share stats calc pending

####existing bug, team defense stats only merging for QBs --> research pending, likely to do with merges in create_master_weekly_player_data()


#Additional possible features creation:
#4. merge weekly player stats with schedule df on game id --> pending conversion
#5. merge weekly player stats with injuries df --> pending conversion **also need to evaluate whether it makes sense to identify injuries through tweets/NLP?
#6. merge weekly player stats with depth chart df --> pending conversion
#7. merge weekly player stats with roster df --> pending conversion

#Aggregation of weekly data up until week before each target week:
#8. function to aggregate mean (possibly stdev for some) player stats for full season up until each current week --> pending conversion
#9. function to aggregate opp defense stats for full season up until each current week --> pending conversion

#10.Methodology to set weights for value of previous seasons? --> pending design

#Need to split veterans and rookies into two different models
#11. Function to appropriately split these players --> pending design
#12. Function to compile rookie profiles (sentiment analysis from tweets, combine data, draft value etc) --> pending conversion
#13. Need some kind of approach to weight features from actual NFL production once they start their rookie year? --> pending design

#Training + Testing + Eval Score Regression Models
#14. Training models on pre-2022 --> pending design
#15. Dimensionality reduction/feature selection --> pending design
#16. Evaluation of model(s) --> pending design

#17. Possible training separate classification model (deep learning) to identify booms/busts/safe etc (extra indicator in FE looker). Will label training/test data using diff from scores
  #possible inclusion of tweet sentiment analysis from fantasy experts in season
  #--> pending design

#18. Possible time series forecasting of future player expected points value to classify buy, sell, holds? Probably need to create existing pos rankings (through current week) vs expected future ranks
  #--> pending design


#19. Generate csv for input into looker front end


In [ ]:
# to delete, just setting variables here for testing

seasons = [2020,2021,2022]

pbp_cols = ['play_id', 'game_id','home_team', 'away_team', 'season_type', 'week','posteam', 'posteam_type','defteam','side_of_field', 'yardline_100','goal_to_go',
            'yards_gained','play_type',
            'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble','pass_length', 'pass_location', 'air_yards','sack','yards_after_catch', 'run_location', 'run_gap',
            'td_team', 'td_player_name','td_player_id','rush_attempt', 'pass_attempt','touchdown', 'pass_touchdown','rush_touchdown', 'return_touchdown',
            'fumble','complete_pass', 'passer_player_id', 'passer_player_name', 'passing_yards', 'receiver_player_id', 'receiver_player_name', 'receiving_yards', 
            'rusher_player_id', 'rusher_player_name', 'rushing_yards','fumbled_1_player_id', 'fumbled_1_player_name','success','pass','rush','play', 'passer_id',
            'rusher_id', 'receiver_id','fantasy_player_name', 'fantasy_player_id']

wk_cols = ['player_id', 'player_name', 'player_display_name', 'position','recent_team', 'season', 'week', 'season_type', 'completions', 'attempts',
           'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards','sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 
           'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs','receptions', 'targets', 'receiving_yards',
           'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch',
           'receiving_first_downs', 'receiving_2pt_conversions','target_share','air_yards_share','fantasy_points', 'fantasy_points_ppr']

In [ ]:
# to delete, just here for testing

pbp_data = nfl.import_pbp_data(years=seasons, columns=pbp_cols, downcast=False, cache=False, alt_path=None)
pbp_data = pbp_data[pbp_cols]

2020 done.
2021 done.
2022 done.


In [ ]:
# to delete, just here for testing

pbp_data.sample(10)

,play_id,game_id,home_team,away_team,season_type,week,side_of_field,yardline_100,goal_to_go,yards_gained,...,fumbled_1_player_name,success,pass,rush,play,passer_id,rusher_id,receiver_id,fantasy_player_name,fantasy_player_id
111351,1326.0,2020_04_MIN_HOU,HOU,MIN,REG,4,HOU,35.0,0.0,0.0,...,None,0.0,0.0,0.0,0.0,None,None,None,None,None
98777,1271.0,2021_19_LV_CIN,CIN,LV,POST,19,CIN,10.0,0.0,0.0,...,None,0.0,1.0,0.0,1.0,00-0031280,None,00-0026189,D.Jackson,00-0026189
12527,3631.0,2022_05_IND_DEN,DEN,IND,REG,5,IND,73.0,0.0,0.0,...,None,0.0,1.0,0.0,1.0,00-0026143,None,00-0037755,J.Woods,00-0037755
36249,4189.0,2022_14_NE_ARI,ARI,NE,REG,14,None,NaN,0.0,NaN,...,None,0.0,0.0,0.0,0.0,None,None,None,None,None
81768,496.0,2021_12_SEA_WAS,WAS,SEA,REG,12,SEA,11.0,0.0,5.0,...,None,1.0,0.0,1.0,1.0,None,00-0036328,None,A.Gibson,00-0036328
34068,3275.0,2022_13_SEA_LA,LA,SEA,REG,13,SEA,35.0,0.0,0.0,...,None,0.0,0.0,0.0,0.0,None,None,None,None,None
79053,2794.0,2021_11_NYG_TB,TB,NYG,REG,11,NYG,15.0,0.0,0.0,...,None,1.0,0.0,0.0,0.0,None,None,None,None,None
68673,1149.0,2021_07_NO_SEA,SEA,NO,REG,7,SEA,45.0,0.0,0.0,...,None,0.0,0.0,1.0,1.0,None,00-0033906,None,A.Kamara,00-0033906
6760,120.0,2022_03_GB_TB,TB,GB,REG,3,GB,42.0,0.0,0.0,...,None,0.0,0.0,1.0,1.0,None,00-0033856,None,L.Fournette,00-0033856
95317,2812.0,2021_17_NYG_CHI,CHI,NYG,REG,17,CHI,43.0,0.0,3.0,...,None,0.0,0.0,1.0,1.0,None,00-0034844,None,S.Barkley,00-0034844


# Define Functions to build modular pipeline

In [ ]:
#extract season data from nfl_data_py, both inputs taken as list (int of each season, list of columns)
#assumption is that current season will be updated after each week of games, if not sooner
#daily pipeline will repeatedly extract current season

def extract_all_season_data_dfs(seasons, pb_columns, wk_columns):
  pbp_data = nfl.import_pbp_data(years=seasons, downcast=False, cache=False, alt_path=None)
  pbp_data = pbp_data[pb_columns] #limit to relevant columns, parameter for method call in nfl_data_py appears to not work
  pbp_data = pbp_data[(pbp_data['play'] == 1.0) & (pbp_data['season_type']=='REG')] #only non-special teams plays (pass or rush) and for regular season weeks (fantasy season)
  pbp_data.fillna(np.nan, inplace=True) #replace None types
  pbp_data['season'] = [int(g[:4]) for g in pbp_data['game_id']]

  week_data = nfl.import_weekly_data(years=seasons, columns=wk_columns, downcast=False) #limit to relevant columns
  week_data = week_data[(week_data['position'].isin(['QB','RB','WR','TE'])&(week_data['season_type']=='REG'))] #only fantasy relevant positions & for regular season weeks (fantasy season)


  sched_data = nfl.import_schedules(years=seasons) #contains game meta data

  rosters_data = nfl.import_rosters(years=seasons) #contains team level roster info and player profile data - might be a good way to split rookies, but unclear if updated in season

  injuries_data = nfl.import_injuries(years=seasons) #contains injury reports

  combine_data = nfl.import_combine_data(years=seasons, positions=['QB','RB','WR','TE']) #only fantasy relevant positions

  depth_data = nfl.import_depth_charts(years=seasons) #contains depth chart information

  latest_season = max(seasons) # to identify latest season in data

  return pbp_data, week_data, sched_data, rosters_data, injuries_data, combine_data, depth_data, latest_season

In [ ]:
# need to create some additional counting stats at the play level for additional potential features

def additional_pbp_counting_stats(pbp_df):
  pbp_df['compl_passes_20_yds'] = np.where(((pbp_df['complete_pass'] == 1.0) & (pbp_df['passing_yards'] >= 20.0)), 1, 0)
  pbp_df['compl_passes_40_yds'] = np.where(((pbp_df['complete_pass'] == 1.0) & (pbp_df['passing_yards'] >= 40.0)), 1, 0)
  pbp_df['compl_passes_60_yds'] = np.where(((pbp_df['complete_pass'] == 1.0) & (pbp_df['passing_yards'] >= 60.0)), 1, 0)
  pbp_df['pass_att_20_yds'] = np.where(((pbp_df['pass_attempt'] == 1.0) & (pbp_df['air_yards'] >= 20.0)), 1, 0)
  pbp_df['pass_att_40_yds'] = np.where(((pbp_df['pass_attempt'] == 1.0) & (pbp_df['air_yards'] >= 40.0)), 1, 0)
  pbp_df['pass_att_60_yds'] = np.where(((pbp_df['pass_attempt'] == 1.0) & (pbp_df['air_yards'] >= 60.0)), 1, 0)
  pbp_df['pass_att_goal'] = np.where(((pbp_df['pass_attempt']==1.0) & (pbp_df['yardline_100'] <= 5)), 1, 0)
  pbp_df['pass_att_red'] = np.where(((pbp_df['pass_attempt']==1.0) & (pbp_df['yardline_100'] <= 20)), 1, 0)
  pbp_df['rush_20_yds'] = np.where(((pbp_df['rush_attempt']==1.0) & (pbp_df['rushing_yards'] >= 20.0)), 1, 0)
  pbp_df['rush_40_yds'] = np.where(((pbp_df['rush_attempt']==1.0) & (pbp_df['rushing_yards'] >= 40.0)), 1, 0)
  pbp_df['rush_60_yds'] = np.where(((pbp_df['rush_attempt']==1.0) & (pbp_df['rushing_yards'] >= 60.0)), 1, 0)
  pbp_df['goal_rush'] = np.where(((pbp_df['rush_attempt']==1.0) & (pbp_df['yardline_100'] <= 5)), 1, 0)
  pbp_df['red_rush'] = np.where(((pbp_df['rush_attempt']==1.0) & (pbp_df['yardline_100'] <= 20)), 1, 0)
  return pbp_df


Play By Play (PBP) Metrics not included in weekly data:

Player Level:  Goal line rushes, Red Zone Rushes, Red Zone Targets, Red Zone Pass Att, Deep Pass Att, Deep Compl Pass Attempts, Deep Targets, Big Plays

Team Offense Level: Total Valuable Plays, Total Rushes, Total Passes, Goal line rushes, Red Zone Rushes, Red Zone Targets, Red Zone Pass Att, Deep Pass Att, Deep Compl Pass Attempts, Deep Targets, Big Plays

Team Defense Level: Yards Allowed, Pass Attempts Allowed, Completions Allowed, Rushes allowed, YPA allowed, YPC allowed, YAC allowed, Long passes allowed, Long rushes allowed

In [ ]:
# need to collapse all pbp data to player id level, offense team level, and defense team level all at game level -- returns multiple dfs

def aggregate_pbp_data(pbp_df):
  player_game_df_pass = pbp_df.groupby(['game_id','season','week','passer_id']).agg({'compl_passes_20_yds':'sum','compl_passes_40_yds':'sum','compl_passes_60_yds':'sum',
                                                                                  'pass_att_20_yds':'sum','pass_att_40_yds':'sum','pass_att_60_yds':'sum',
                                                                                  'pass_att_goal':'sum','pass_att_red':'sum','sack':'sum'}).reset_index()
  player_game_df_rush = pbp_df.groupby(['game_id','season','week','rusher_id']).agg({'rush_20_yds':'sum','rush_40_yds':'sum','rush_60_yds':'sum',
                                                                                  'goal_rush':'sum','red_rush':'sum'}).reset_index()
  player_game_df_receive = pbp_df.groupby(['game_id','season','week','receiver_id']).agg({'compl_passes_20_yds':'sum','compl_passes_40_yds':'sum','compl_passes_60_yds':'sum',
                                                                                    'pass_att_20_yds':'sum','pass_att_40_yds':'sum','pass_att_60_yds':'sum',
                                                                                    'pass_att_goal':'sum','pass_att_red':'sum'}).reset_index()
  player_game_df_receive.columns = ['game_id','season','week','receiver_id','catches_20_yards','catches_40_yards','catches_60_yards',
                                    'targets_20_yards','targets_40_yards','targets_60_yards','targets_goal','targets_red'] #rename receiving stats to avoid confusion with passing stats


  offense_game_df = pbp_df.groupby(['game_id','season','week','posteam']).agg({'compl_passes_20_yds':'sum','compl_passes_40_yds':'sum','compl_passes_60_yds':'sum',
                                                          'pass_att_20_yds':'sum','pass_att_40_yds':'sum','pass_att_60_yds':'sum',
                                                          'pass_att_goal':'sum','pass_att_red':'sum','goal_rush':'sum','red_rush':'sum',
                                                           'pass_attempt':'sum','rush_attempt':'sum'}).reset_index()
                      
  offense_game_df.columns = ['game_id','season','week','posteam','team_20_yard_compl_o','team_40_yard_compl_o','team_60_yard_compl_o',
                             'team_20_yard_pass_att_o','team_40_yard_pass_att_o','team_60_yard_pass_att_o','team_goal_pass_att_o',
                             'team_red_pass_att_o','team_goal_rush_att_o','team_red_rush_att_o','team_pass_att_o','team_rush_att_o'
                            ]             

  defense_game_df = pbp_df.groupby(['game_id','season','week','defteam']).agg({'compl_passes_20_yds':'sum','compl_passes_40_yds':'sum','compl_passes_60_yds':'sum',
                                                          'pass_att_20_yds':'sum','pass_att_40_yds':'sum','pass_att_60_yds':'sum',
                                                          'pass_att_goal':'sum','pass_att_red':'sum','goal_rush':'sum','red_rush':'sum',
                                                           'pass_attempt':'sum','complete_pass':'sum','rush_attempt':'sum','pass_touchdown':'sum',
                                                           'rush_touchdown':'sum','passing_yards':'sum','rushing_yards':'sum',
                                                           'sack':'sum','yards_after_catch':'sum'}).reset_index()
  

  defense_game_df['ypa_allowed'] = defense_game_df['passing_yards']/defense_game_df['pass_attempt'] # defense yards per attempt allowed
  defense_game_df['ypc_allowed'] = defense_game_df['rushing_yards']/defense_game_df['rush_attempt'] # defense yards per carry allowed
  defense_game_df['yac_per_compl_allowed'] = defense_game_df['yards_after_catch']/defense_game_df['complete_pass'] # defense yards after catch per catch allowed

  defense_game_df.columns = ['game_id','season','week','defteam','team_20_yard_compl_d','team_40_yard_compl_d','team_60_yard_compl_d',
                             'team_20_yard_pass_att_d','team_40_yard_pass_att_d','team_60_yard_pass_att_d','team_goal_pass_att_d',
                             'team_red_pass_att_d','team_goal_rush_att_d','team_red_rush_att_d','team_pass_att_d','team_compl_allowed',
                             'team_rush_att_d','pass_tds_allowed','rush_tds_allowed','passing_yards_allowed','rushing_yards_allowed','sacks_d','yac_allowed',
                             'ypa_allowed','ypc_allowed','yac_per_compl_allowed'
                            ]

  return player_game_df_pass, player_game_df_rush, player_game_df_receive, offense_game_df, defense_game_df                                                      

In [ ]:
# need to merge new player level stats & offense + defense team level stats, to weekly stats -- return master player-level weekly df

def create_master_weekly_player_data(week_data, player_game_df_pass, player_game_df_rush, player_game_df_receive, offense_game_df, defense_game_df):
  master_wk_df = pd.merge(week_data,player_game_df_pass,how='left', left_on=['player_id','season','week'], right_on=['passer_id','season','week'])
  master_wk_df = pd.merge(master_wk_df,player_game_df_rush,how='left', left_on=['player_id','season','week','game_id'], right_on=['rusher_id','season','week','game_id'])
  master_wk_df = pd.merge(master_wk_df,player_game_df_receive,how='left', left_on=['player_id','season','week','game_id'], right_on=['receiver_id','season','week','game_id'])
  master_wk_df = pd.merge(master_wk_df,offense_game_df,how='left', left_on=['season','week','recent_team','game_id'], right_on=['season','week','posteam','game_id'])

  master_wk_df = pd.merge(master_wk_df,defense_game_df,how='left', left_on=['season','week','game_id'], right_on=['season','week','game_id'])

  master_wk_df = master_wk_df[master_wk_df['posteam'] != master_wk_df['defteam']]
  master_wk_df.drop_duplicates(inplace=True)

  return master_wk_df


# Call Functions to Test Pipeline

In [ ]:
pbp_data, week_data, sched_data, rosters_data, injuries_data, combine_data, depth_data, latest_season = extract_all_season_data_dfs(seasons, pbp_cols, wk_cols)

player_game_df_pass, player_game_df_rush, player_game_df_receive, offense_game_df, defense_game_df = aggregate_pbp_data(additional_pbp_counting_stats(pbp_data))

create_master_weekly_player_data(week_data, player_game_df_pass, player_game_df_rush, player_game_df_receive, offense_game_df, defense_game_df).sample(15)

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'recent_team', 'season', 'week', 'season_type', 'completions',
       'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'sacks',
       'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs',
       'receiving_2pt_conversions', 'target_share', 'air_yards_share',
       'fantasy_points', 'fantasy_points_ppr', 'game_id', 'passer_id',
       'compl_passes_20_yds', 'compl_passes_40_yds', 'compl_passes_60_yds',
       'pass_att_20_yds', 'pass_att_40_yds', 'pass_att_60_yds',
       'pass_att_goal', 'pass_att_red', 'sack', 'rusher_id', 'rush_20_yds

,player_id,player_name,player_display_name,position,recent_team,season,week,season_type,completions,attempts,...,team_rush_att_d,pass_tds_allowed,rush_tds_allowed,passing_yards_allowed,rushing_yards_allowed,sacks_d,yac_allowed,ypa_allowed,ypc_allowed,yac_per_compl_allowed
14784,00-0034487,T.Sherfield,Trent Sherfield,WR,MIA,2022,15,REG,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10820,00-0036255,D.Mims,Denzel Mims,WR,NYJ,2021,15,REG,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13270,00-0032436,J.Driskel,Jeff Driskel,QB,HOU,2022,18,REG,2,2,...,23.0,3.0,0.0,311.0,59.0,1.0,120.0,7.404762,2.565217,5.000000
8413,00-0033555,M.Hollins,Mack Hollins,WR,MIA,2021,15,REG,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1610,00-0031598,N.Boyle,Nick Boyle,TE,BAL,2020,1,REG,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,00-0020531,D.Brees,Drew Brees,QB,NO,2020,8,REG,31,41,...,29.0,2.0,0.0,280.0,123.0,1.0,169.0,6.666667,4.241379,5.451613
40,00-0020531,D.Brees,Drew Brees,QB,NO,2020,3,REG,29,36,...,20.0,3.0,0.0,288.0,122.0,2.0,163.0,7.783784,6.100000,5.620690
5672,00-0036357,H.Ruggs,Henry Ruggs,WR,LV,2020,17,REG,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11654,00-0036862,B.Skowronek,Ben Skowronek,WR,LA,2021,21,POST,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2619,00-0033382,A.Carr,Austin Carr,WR,NO,2020,17,REG,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
